In [1]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

In [2]:
conf=SparkConf()
# conf.set("spark.executor.memory", "2g")
# conf.set("spark.driver.memory", "12g")
# conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/07 20:41:03 WARN Utils: Your hostname, office-preacher resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp4s0)
23/12/07 20:41:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/07 20:41:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/07 20:41:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/preacher/anaconda3/envs/delta_run/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
matches_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
])

mdw_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
    StructField("match_type", StringType(), True)
])

In [136]:
matche_data = [
    ["new1<>203359001<>homedepot<>homedepot_compNoSugg1p1<>g4103325<>bcg<>bcg","new1<>203359001<>homedepot<>homedepot","homedepot<>homedepot","compNoSugg1p1<>g4103325<>bcg<>bcg", "bcg<>bcg","1P"],
    ["new2<>100341666<>homedepot<>homedepot_compNoSugg1p2<>109589<>bcg<>bcg","new2<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg1p2<>109589<>bcg<>bcg", "bcg<>bcg","1P"],
    
    ["1<>203370169<>homedepot<>homedepot_compYes1pExact<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pExact<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    ["2<>100670057<>homedepot<>homedepot_compYes1pSim<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pSim<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    ["3<>100341666<>homedepot<>homedepot_compYes3pSim<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compYes3pSim<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["4<>53409<>petcarerx<>petcarerx_compYes3pExact<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compYes3pExact<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    
    ["new3<>53409<>petcarerx<>petcarerx_compNoSugg3p1<>147664<>chewy<>chewy","new3<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNoSugg3p1<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["new4<>100341666<>homedepot<>homedepot_compNoSugg3p2<>109589<>bcg<>bcg","new4<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg3p2<>109589<>bcg<>bcg", "bcg<>bcg","3P"],

    ["1<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    ["2<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    ["3<>100341666<>homedepot<>homedepot_compNo3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg3p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNo3pExactSugg3p<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

    [ "1<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967446<>bcg<>bcg","bcg<>bcg","3P"],
    [ "2<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "3<>100341666<>homedepot<>homedepot_compNo3pSimSugg1p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pExactSugg1p<>109589<>bcg<>bcg","bcg<>bcg","1P"],
    [ "4<>53409<>petcarerx<>petcarerx_compNo3pExactSugg1p<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compNo3pExactSugg1p<>147664<>chewy<>chewy", "chewy<>chewy","1P"],
    
]
matches = spark.createDataFrame(matche_data, schema=matches_schema)

mdw_data = [
    [ "1<>203370169<>homedepot<>homedepot_compYes1pExact<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pExact<>967446<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "2<>100670057<>homedepot<>homedepot_compYes1pSim<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pSim<>967493<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "3<>100341666<>homedepot<>homedepot_compYes3pSim<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compYes3pSim<>109589<>bcg<>bcg","bcg<>bcg","3P","similar"],
    [ "4<>53409<>petcarerx<>petcarerx_compYes3pExact<>147664<>chewy<>chewy","4<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","compYes3pExact<>147664<>chewy<>chewy", "chewy<>chewy","3P","exact"],
]
mdw = spark.createDataFrame(mdw_data, schema=mdw_schema)

In [137]:
matches.show()
mdw.show()

+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|             pair_id|       base_sku_uuid|   base_source_store|       comp_sku_uuid|comp_source_store|seller_type|
+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|new1<>203359001<>...|new1<>203359001<>...|homedepot<>homedepot|compNoSugg1p1<>g4...|         bcg<>bcg|         1P|
|new2<>100341666<>...|new2<>100341666<>...|homedepot<>homedepot|compNoSugg1p2<>10...|         bcg<>bcg|         1P|
|1<>203370169<>hom...|1<>203370169<>hom...|homedepot<>homedepot|compYes1pExact<>9...|         bcg<>bcg|         1P|
|2<>100670057<>hom...|2<>100670057<>hom...|homedepot<>homedepot|compYes1pSim<>967...|         bcg<>bcg|         1P|
|3<>100341666<>hom...|3<>100341666<>hom...|homedepot<>homedepot|compYes3pSim<>109...|         bcg<>bcg|         3P|
|4<>53409<>petcare...|4<>53409<>petcare...|petcarerx<>petcarerx|compYes3

In [110]:
def combine_dfs(df_list: list[DataFrame]) -> DataFrame:
    print("Combining dfs")
    if df_list is None:
        return None
    
    df_list = [df for df in df_list if df is not None]
    if len(df_list) == 0:
        return None
    if len(df_list) == 1:
        return df_list[0]
    combined_df = reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), df_list)
    return combined_df

In [132]:
from pyspark.sql.window import Window

class Match_Pruner:
    def __init__(self, mdw: DataFrame, match_suggestion: DataFrame, config: dict) -> None:
        self.mdw = mdw
        self.match_suggestion = match_suggestion
        self.config = config

    def prune_matches(self):
        self.mdw, match_suggestion = self.preprocessing_matches(self.mdw, self.match_suggestion)
        # filtered_match_suggestion, _1p_3p_pruned_match_suggestion = self.prune_1p_3p_match_suggestions(match_suggestion)
        # print("filtered_match_suggestion")
        # filtered_match_suggestion.show()
        # if _1p_3p_pruned_match_suggestion is not None:
        #     print("_1p_3p_pruned_match_suggestion")
        #     _1p_3p_pruned_match_suggestion.show()
        # unaudited_base_match_suggestion, audited_base_product_matches = self.separate_audited_unaudited_base_product_matches(filtered_match_suggestion, self.mdw)
        # unaudited_base_match_suggestion, audited_base_product_matches = self.separate_audited_unaudited_base_product_matches(match_suggestion, self.mdw)
        # print("unaudited_base_match_suggestion")
        # unaudited_base_match_suggestion.show()
        # print("audited_base_product_matches")
        # audited_base_product_matches.show()

        similar_matches, exact_matches, audited_base_product_unaudited_matches = self.separate_audited_and_unaudited_matches(match_suggestion, self.mdw)
        print("similar_matches")
        similar_matches.show()
        print("exact_matches")
        exact_matches.show()
        print("audited_base_product_unaudited_matches")
        audited_base_product_unaudited_matches.show()

        match_type_pruned_suggestion, unaudited_match_type_suggestion = self.prune_match_based_on_match_type(similar_matches, exact_matches, audited_base_product_unaudited_matches)
        if match_type_pruned_suggestion is not None:
            print("match_type_pruned_suggestion")
            match_type_pruned_suggestion.show()
        if unaudited_match_type_suggestion is not None:
            print("unaudited_match_type_suggestion")
            unaudited_match_type_suggestion.show()

        # pruned_suggestion = combine_dfs([_1p_3p_pruned_match_suggestion, match_type_pruned_suggestion])
        # match_suggestion = combine_dfs([unaudited_base_match_suggestion, unaudited_match_type_suggestion])
        pruned_suggestion = combine_dfs([ match_type_pruned_suggestion])
        match_suggestion = combine_dfs([unaudited_match_type_suggestion])

        return match_suggestion, pruned_suggestion

    def preprocessing_matches(self, mdw: DataFrame, match_suggestion: DataFrame):
        match_suggestion = match_suggestion.withColumn("seller_type", lower(col("seller_type")))
        mdw = mdw.withColumn("seller_type", lower(col("seller_type")))\
            .withColumn("match_type", when( col("match_type").isNotNull(), lower(col("match_type")) ).otherwise(col("match_type")) )

        return mdw, match_suggestion
    
    def prune_1p_3p_match_suggestions(self, match_suggestion: DataFrame):
        filtered_match_suggestion = None
        pruned_match_suggestion = None
        if self.config['seller_type'] == '1p':
            filtered_match_suggestion = match_suggestion.filter(col('seller_type') == '1p')
            pruned_match_suggestion = match_suggestion.filter(col('seller_type') == '3p')
            pruned_match_suggestion = pruned_match_suggestion.withColumn("misc_info", lit("pruned_by_seller_type_1p"))
        elif self.config['seller_type'] == '3p':
            filtered_match_suggestion = match_suggestion.filter(col('seller_type') == '3p')
            pruned_match_suggestion = match_suggestion.filter(col('seller_type') == '1p')
            pruned_match_suggestion = pruned_match_suggestion.withColumn("misc_info", lit("pruned_by_seller_type_3p"))
        else:
            filtered_match_suggestion = match_suggestion
        return filtered_match_suggestion, pruned_match_suggestion
    
    def separate_audited_unaudited_base_product_matches(self, match_suggestion:DataFrame, mdw:DataFrame):
        mdw = mdw.select("base_sku_uuid")
        unaudited_base_product_matches = match_suggestion.join(mdw, 'base_sku_uuid', 'left_anti')
        audited_base_product_matches = match_suggestion.join(mdw, 'base_sku_uuid', 'inner')
        return unaudited_base_product_matches, audited_base_product_matches
    
    def separate_audited_and_unaudited_matches(self, audited_base_product_matches:DataFrame, mdw:DataFrame):
        print("separate_audited_and_unaudited_matches")
        # audited_base_product_matches.show(truncate=False)
        audited_matches = audited_base_product_matches.select("pair_id").join(mdw, 'pair_id', 'inner')
        # audited_matches.show(truncate=False)
        window_spec = Window.partitionBy("base_source_store")

        similar_matches = audited_matches.filter(col('match_type').isin(['equivalent', 'similar']))
        similar_matches = similar_matches.withColumn("audited_match_comp_seller_type", concat_ws("_", collect_set(col("seller_type")).over(window_spec)))
        similar_matches = similar_matches.withColumn("misc_info", concat(lit("similar_match_"), col("pair_id")) ).\
            select("base_sku_uuid", "audited_match_comp_seller_type", "misc_info")
                                        
               
        exact_matches = audited_matches.filter(col('match_type').isin(['exact']))
        exact_matches = exact_matches.withColumn("audited_match_comp_seller_type", concat_ws("_", collect_set(col("seller_type")).over(window_spec)))
        exact_matches = exact_matches.withColumn("misc_info", concat(lit("exact_match_"), col("pair_id")) ).\
                        select("base_sku_uuid", "audited_match_comp_seller_type","misc_info")

        audited_base_product_unaudited_matches = audited_base_product_matches.join(audited_matches.select("pair_id"), 'pair_id', 'left_anti')
        return similar_matches, exact_matches, audited_base_product_unaudited_matches

    def prune_match_based_on_match_type(self, similar_matches:DataFrame , exact_matches:DataFrame , audited_base_product_unaudited_matches:DataFrame ):
        # audited_base_product_unaudited_matches contains only unaudited matches 
        pruned_suggestion = None
        match_suggestion = None
        if self.config['match_type'] == 'exact':
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
            if self.config['seller_type'] == '1p':
                pruned_suggestion = audited_base_product_unaudited_matches.filter(col("misc_info").isNotNull() | (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter(col("misc_info").isNull() & (col("seller_type") == '1p')).drop("misc_info")
            elif self.config['seller_type'] == '3p':
                # audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter(col("misc_info").isNotNull() | (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                    match_suggestion = audited_base_product_unaudited_matches.filter(col("misc_info").isNull() & (col("seller_type") == '3p')).drop("misc_info")
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter(col("seller_type") == '1p').withColumn("misc_info", lit("seller_type_config_3p"))
                    match_suggestion = audited_base_product_unaudited_matches.filter(col("seller_type") == '3p')

            elif self.config['seller_type'] == '1p_or_3p':
                # audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches
                    match_suggestion = None
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("audited_match_comp_seller_type").contains("1p") ) & ( col('comp_seller_type') == '1p' ) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( ~((col("audited_match_comp_seller_type").contains("1p") ) & ( col('comp_seller_type') == '1p' )) ) 

            elif self.config['seller_type'] == '1p_over_3p':
                # audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( ~((col("audited_match_comp_seller_type") == "3p") & ( col('comp_seller_type') == '1p' )) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( (col("audited_match_comp_seller_type") == "3p") & ( col('comp_seller_type') == '1p' ) )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("audited_match_comp_seller_type").contains("1p") ) & ( col('comp_seller_type') == '1p' ) )
                    match_suggestion = audited_base_product_unaudited_matches.filter( ~((col("audited_match_comp_seller_type").contains("1p") ) & ( col('comp_seller_type') == '1p' )) )

        elif self.config['match_type'] == 'similar' or self.config['match_type'] == 'exact_or_similar':
            #  audited_base_product_unaudited_matches only contain 1p match suggestion
            exact_and_similar_match = exact_matches.union(similar_matches)
            if self.config['seller_type'] == '1p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match.filter(col("audited_match_comp_seller_type").contains("1p")), "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("match_type") == 'exact')  )
                    match_suggestion = audited_base_product_unaudited_matches.filter( ~(col("match_type") == 'exact') )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = None
                    match_suggestion = audited_base_product_unaudited_matches
            elif self.config['seller_type'] == '3p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match.filter(col("audited_match_comp_seller_type").contains("3p")), "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("match_type") == 'exact')  )
                    match_suggestion = audited_base_product_unaudited_matches.filter( ~( (col("match_type") == 'exact')  ) )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = None
                    match_suggestion = audited_base_product_unaudited_matches
            elif self.config['seller_type'] == '1p_or_3p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( col("match_type") == 'exact' )
                    match_suggestion = audited_base_product_unaudited_matches.filter( ~( col("match_type") == 'exact' ) )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = None
                    match_suggestion = audited_base_product_unaudited_matches
            elif self.config['seller_type'] == '1p_over_3p':
                audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match, "base_sku_uuid", "left")
                if self.config['cardinality'] == '1':
                    condition = ( col("match_type") == 'exact') & (
                        (col('audited_match_comp_seller_type').contains('1p') )  |
                        ((col('audited_match_comp_seller_type') == '3p') & (col('comp_seller_type') == '3p'))
                    )
                    pruned_suggestion = audited_base_product_unaudited_matches.filter( condition )
                    match_suggestion = audited_base_product_unaudited_matches.filter( ~(condition) )
                if self.config['cardinality'] == 'n':
                    pruned_suggestion = None
                    match_suggestion = audited_base_product_unaudited_matches
        return pruned_suggestion, match_suggestion
        
    

In [138]:
config = {
    "seller_type" : "1p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

separate_audited_and_unaudited_matches
similar_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|2<>100670057<>hom...|                         1p_3p|similar_match_2<>...|
|3<>100341666<>hom...|                         1p_3p|similar_match_3<>...|
+--------------------+------------------------------+--------------------+

exact_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|1<>203370169<>hom...|                            1p|exact_match_1<>20...|
|4<>53409<>petcare...|                            3p|exact_match_4<>53...|
+--------------------+------------------------------+--------------------+

audited_base_product_unaudite

In [139]:
config = {
    "seller_type" : "1p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

separate_audited_and_unaudited_matches
similar_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|2<>100670057<>hom...|                         1p_3p|similar_match_2<>...|
|3<>100341666<>hom...|                         1p_3p|similar_match_3<>...|
+--------------------+------------------------------+--------------------+

exact_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|1<>203370169<>hom...|                            1p|exact_match_1<>20...|
|4<>53409<>petcare...|                            3p|exact_match_4<>53...|
+--------------------+------------------------------+--------------------+

audited_base_product_unaudite

In [140]:
config = {
    "seller_type" : "3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

n = 1
matche_data = [
    ["0000346318<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","0000346318<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
]

separate_audited_and_unaudited_matches
similar_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|2<>100670057<>hom...|                         1p_3p|similar_match_2<>...|
|3<>100341666<>hom...|                         1p_3p|similar_match_3<>...|
+--------------------+------------------------------+--------------------+

exact_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|1<>203370169<>hom...|                            1p|exact_match_1<>20...|
|4<>53409<>petcare...|                            3p|exact_match_4<>53...|
+--------------------+------------------------------+--------------------+

audited_base_product_unaudite

In [141]:
config = {
    "seller_type" : "3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
match_suggestion.show(truncate=False)
pruned_suggestion.show(truncate=False)

matche_data = [
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_0000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","0000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

]

separate_audited_and_unaudited_matches
similar_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|2<>100670057<>hom...|                         1p_3p|similar_match_2<>...|
|3<>100341666<>hom...|                         1p_3p|similar_match_3<>...|
+--------------------+------------------------------+--------------------+

exact_matches
+--------------------+------------------------------+--------------------+
|       base_sku_uuid|audited_match_comp_seller_type|           misc_info|
+--------------------+------------------------------+--------------------+
|1<>203370169<>hom...|                            1p|exact_match_1<>20...|
|4<>53409<>petcare...|                            3p|exact_match_4<>53...|
+--------------------+------------------------------+--------------------+

audited_base_product_unaudite

In [112]:
config = {
    "seller_type" : "1p_3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
match_suggestion, pruned_suggestion = pruner.prune_matches()
if match_suggestion is not None:
    match_suggestion.show(truncate=False)
else:
    print("match_suggestion is empty")

if pruned_suggestion is not None:
    pruned_suggestion.show(truncate=False)
else:
    print("pruned_suggestion is empty")

matche_data = [
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_0000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","0000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

]

filtered_match_suggestion
+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|             pair_id|       base_sku_uuid|   base_source_store|       comp_sku_uuid|comp_source_store|seller_type|
+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|0000346280418<>20...|0000346280418<>20...|homedepot<>homedepot|346280418<>g41033...|         bcg<>bcg|         1p|
|0000346318654<>10...|0000346318654<>10...|homedepot<>homedepot|000346318654<>109...|         bcg<>bcg|         1p|
|0000346378252<>20...|0000346378252<>20...|homedepot<>homedepot|000346378252<>967...|         bcg<>bcg|         1p|
|0000346383867<>10...|0000346383867<>10...|homedepot<>homedepot|000346383867<>967...|         bcg<>bcg|         1p|
|00003463186541<>1...|00003463186541<>1...|homedepot<>homedepot|000346318654<>109...|         bcg<>bcg|         3p|
|0000000000001<>53...|0000000000001<>53...|pet